In [2]:
#1. import libraries
import pandas as pd
from sqlalchemy import create_engine, text
import json
import psycopg2
import matplotlib.pyplot as plt

In [3]:
#2. Load the database configuration
with open (r"C:\Users\user\Documents\Portfolio_Vista\Retail_Fashion\scripts\db_config.json","r") as file:
    config = json.load(file)

username = config["username"]
password = config["password"]
host = config["host"]
port = config["port"]
database = config["database"]

print ('Succesful loaded the credential!')

Succesful loaded the credential!


In [4]:
#3. Connect to PostgreSQL
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')
print (f'Connected to database {database}')

Connected to database shoes


In [10]:
#4. Query
#1. Which products sell in high quantities and need restocking?
query = """
select
	product_name,
	sum(quantity) as total_sold
from
	sales_shoes
group by
	product_name
order by
	total_sold desc
limit 5;
"""
df_1 = pd.read_sql(query, engine)
display (df_1)

,product_name,total_sold
0,Off-White Hoodie,143.0
1,Nike Tech Fleece,140.0
2,Yeezy Boost 350,119.0
3,Puma Joggers,116.0
4,Jordan 1 High,113.0


In [11]:
#2. Are there products with low sales that could be discounted or removed from inventory? (based on total sold)
query = """
select
	product_name,
	sum (quantity) as total_sold
from
	sales_shoes
group by
	product_name
order by
	total_sold
limit 3;
"""
df_2 = pd.read_sql(query, engine)
display (df_2)

,product_name,total_sold
0,Fear of God Essentials Tee,80.0
1,New Era Cap,88.0
2,Adidas Ultraboost,91.0


In [12]:
#3. How does seasonality affect product sales (e.g., monthly or quarterly trends)?
query = """
with monthly_trend as (
	select 
		extract (month from date) as month,
		extract (year from date) as year,
		extract (quarter from date) as quarter,
		sum(amount) as monthly_revenue
	from 
		sales_shoes
	group by 
		month,
		year,
		quarter
	order by 
		month,
		year,
		quarter
	)
select
	*,
	sum (monthly_revenue) over (partition by quarter order by quarter) as quarterly_revenue
from 
	monthly_trend
group by 
	month,
	year,
	quarter,
	monthly_revenue;
"""
df_3 = pd.read_sql(query, engine)
display (df_3)

,month,year,quarter,monthly_revenue,quarterly_revenue
0,1.0,2022.0,1.0,23067.92,72504.64
1,2.0,2022.0,1.0,20393.54,72504.64
2,3.0,2022.0,1.0,29043.18,72504.64
3,4.0,2022.0,2.0,17176.88,63663.54
4,6.0,2022.0,2.0,26089.22,63663.54
5,5.0,2022.0,2.0,20397.44,63663.54
6,7.0,2022.0,3.0,32073.43,56934.89
7,8.0,2022.0,3.0,24861.46,56934.89


In [8]:
#4. Which combinations of product type, brand, and country show consistent high demand?
query = """
select
	product_type,
	brand,
	country,
	sum (amount) as total_revenue
from 
	sales_shoes
group by 
	product_type,
	brand,
	country 
order by 
	total_revenue desc;
"""
df_4 = pd.read_sql(query, engine)
display (df_4)

,product_type,brand,country,total_revenue
0,Joggers,Puma,Japan,8870.48
1,Sneakers,Nike,Japan,8089.11
2,Hoodie,Off-White,Canada,7664.08
3,Hoodie,Off-White,USA,6884.06
4,Sneakers,Nike,India,6732.50
5,Sneakers,Nike,Germany,6583.61
6,Sneakers,Nike,USA,6437.67
7,Hoodie,Nike,Germany,6384.73
8,Sneakers,Adidas,UK,6150.77
9,Sneakers,Adidas,Australia,6104.09


In [9]:
data_table = [df_1, df_2, df_3, df_4]
name_table = [1,2,3,4]
titles = [
    "High-Quantity Products for Inventory Replenishment",
    "Low-Sales Products for Discounting or Removal",
    "Seasonal Sales Trends Analysis",
    "High-Demand Combinations of Product Type, Brand, and Country"
]
dir_img = r'C:\Users\user\Documents\Portfolio_Vista\Retail_Fashion\data\inventory_query'
for i,j,k in zip (data_table,name_table,titles):
    fig, ax = plt.subplots(figsize=(len(i.columns) * 3, len(i) * 0.7))
    ax.axis('tight')
    ax.axis('off')

    table = ax.table(
        cellText=i.values,
        colLabels=i.columns,
        loc='center',
        cellLoc='center'
    )
    table.scale(1, 1.5)
    table.auto_set_font_size(False)
    table.set_fontsize(10)

    plt.title(k, fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig(f"{dir_img}\market_insight_{j}.png", bbox_inches='tight', dpi=300)
    plt.close()


<>:27: SyntaxWarning: invalid escape sequence '\m'
<>:27: SyntaxWarning: invalid escape sequence '\m'
C:\Users\user\AppData\Local\Temp\ipykernel_28244\1910001208.py:27: SyntaxWarning: invalid escape sequence '\m'
  plt.savefig(f"{dir_img}\market_insight_{j}.png", bbox_inches='tight', dpi=300)
